In [2]:
import numpy as np
import pandas as pd

import re
# import re
from os.path import join
import os
import glob

from tools import *

read_prefix = "/data/MIMIC3/"
input_prefix="/data/liu/mimic3/CLAMP_NER/input"
output_prefix="/data/liu/mimic3/CLAMP_NER/ner-attribute/output"
save_csv_prefix="/data/liu/mimic3/CLAMP_NER/ner-attribute/output_csv"
mapping_prefix = "/data/liu/mimic3/MAPPING/"
processed_map_prefix="/data/liu/mimic3/MAPPING/PROCESSED/"
ade_related_prefix="/data/liu/mimic3/CLAMP_NER/ADE_Related/"

In [18]:
# all_disease = read_data("/data/MIMIC3/DIAGNOSES_ICD",dtype={"ICD9_CODE":str})[['ICD9_CODE']].drop_duplicates()
# all_disease.head()
# write2file(all_disease,join(ade_related_prefix,"all_icd9"))

# all_disease = read_data(join(ade_related_prefix,"all_icd9"),dtype={'ICD9_CODE':str})

In [19]:
# all_drug = pres_df[['NDC']].drop_duplicates()
# all_drug.head()
# write2file(all_drug,join(ade_related_prefix,"all_ndc"))
# all_drug = read_data(join(ade_related_prefix,"all_ndc"),dtype={'NDC':str})

In [2]:
## e.g., Atenolol
drug_ndc='51079075920'

In [4]:
### ndc -> rxnorm
# ndc_cui_map = read_data(join(processed_map_prefix,"rxnorm_ndc_df"),
#                         dtype={"CODE":str})[['CUI_CODE','CODE']].drop_duplicates()
# ndc_cui_map.head()
# # write2file(ndc_cui_map,join(ade_related_prefix,"ndc_cui_map"))


ndc_cui_map = read_data(join(ade_related_prefix,"ndc_cui_map"),dtype={'CODE':str,'CUI_CODE':str})
# ndc_cui_map = ndc_cui_map.set_index('CODE').to_dict()

ndc_cui_map = ndc_cui_map.set_index('CODE')['CUI_CODE'].to_dict()
drug_rxnorm = ndc_cui_map[drug_ndc]
print(drug_rxnorm)

NameError: name 'drug_ndc' is not defined

,NDC,drugbank_id,UMLS_CUI,ICD_CODE,SNOMED_CID,ICD9_CODE
0,00003029305,DB00620,C0000731,789.36,60728008,78936
1,00003049420,DB00620,C0000731,789.36,60728008,78936
2,00009001104,DB00620,C0000731,789.36,60728008,78936
3,00009001201,DB00620,C0000731,789.36,60728008,78936
4,00009003101,DB00620,C0000731,789.36,60728008,78936


In [9]:
### transform ndc to rxnorm for sider
sider_file='/data/liu/adverse_events/ndc_icd9_side_effects'
sider_df = read_data(sider_file, dtype=str)
sider_df.head()

# sider_df.loc[:,'RxNorm']=sider_df['NDC'].apply(lambda x: ndc_cui_map.get(x, None))
print("orignal rows %d"%len(sider_df))
sider_df = sider_df.dropna(subset=['RxNorm'])
print("rows after removing null rxnorm %d"%len(sider_df))
sider_df.head()

write2file(sider_df,ade_related_prefix,"sider4_v1")

In [12]:
sider_fulldf = sider_df[["RxNorm","ICD9_CODE"]].drop_duplicates()
sider_fulldf [sider_fulldf ["RxNorm"]=='197380']

,RxNorm,ICD9_CODE
12454,197380,9951
21190,197380,71690
21191,197380,71990
21192,197380,71698
21193,197380,71680
21194,197380,71998
21195,197380,71688
21196,197380,71988
21197,197380,71980
21198,197380,71989


In [ ]:
def create_folder(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

In [7]:
## create root folder of drug
drug_ade_dir = join(ade_related_prefix, drug_ndc)
# create_folder(drug_ade_dir)

NameError: name 'drug_ndc' is not defined

In [8]:
# drug_single_dir = join(ade_related_prefix,drug_ndc,"single_file")
drug_single_dir = join(ade_related_prefix,"all")

## List of patients (episodes) who were administered the drug
i.  	All diseases and prescriptions for that patient episode (from MIMIC diagnoses and prescription tables)

ii.  	All new diseases and prescriptions in that particular episode (using EPISODE_DIAGNOSIS and EPISODE_PRESCRIPTIONS files created)

### All diseases and prescriptions for that patient episode (from MIMIC diagnoses and prescription tables)

In [20]:
### !!! get full prescription table
# pres_df['RxNorm']=pres_df['NDC'].apply(lambda x: ndc_cui_map.get(x, None))
# write2file(pres_df,join(ade_related_prefix,"pres_df"))
pres_df = read_data(join(ade_related_prefix,"pres_df"),dtype=str)
pres_df=pres_df.dropna(subset=['RxNorm'])

In [21]:
pres_df_count = pres_df.groupby('HADM_ID')['RxNorm'].transform(count)


,SUBJECT_ID,HADM_ID,NDC,RxNorm
0,6,107064,00469061711,108513
1,6,107064,00056017275,855334
2,6,107064,00338055002,1658717
3,6,107064,00054829725,310429
4,6,107064,00056016975,855290


In [20]:

print("{:,d} episodes, {:,d} NDC codes, {:,d} RxNorm codes, {:,d} rows".format(len(pres_df.HADM_ID.unique()), 
                                                                         len(pres_df.NDC.unique()),
                                                                         len(pres_df.RxNorm.unique()),
                                                                         len(pres_df)))
                                                                                

50,208 episodes, 4,137 NDC codes, 2,916 RxNorm codes, 3,549,712 rows


In [20]:
## get code name mark
ndc_rxnorm = pres_df[['NDC',"RxNorm"]].drop_duplicates()

pres_df=read_data(join(read_prefix,'PRESCRIPTIONS'),dtype={'SUBJECT_ID':str,'HADM_ID':str,'NDC':str},\
                  usecols=['NDC','DRUG']).dropna(subset=['NDC'])
ndc_name=pres_df[pres_df['NDC']!='0'].drop_duplicates()
rxnorm_name = left_join(ndc_rxnorm,ndc_name,"NDC")[['RxNorm','DRUG']].drop_duplicates()
rxnorm_name.head()

rxnorm_name_df = rxnorm_name.groupby(['RxNorm'])['DRUG'].apply(lambda x: ', '.join(x)).reset_index()
write2file(rxnorm_name_df,join(ade_related_prefix,"RxNorm_DRUGNAME"))

,RxNorm,DRUG
0,108513,Tacrolimus
1,108513,Tacrolimus Suspension
2,108513,Tacroli
3,108513,tacroli
4,108513,*NF* Tacrolimus Suspension


In [72]:
## only once
## all prescriptions for patients who are administrated the drug
## get original full prescription table from MIMIC III
# pres_df=read_data(join(read_prefix,'PRESCRIPTIONS'),dtype={'SUBJECT_ID':str,'HADM_ID':str,'NDC':str},\
#                   usecols=['SUBJECT_ID','HADM_ID','NDC']).dropna(subset=['NDC'])
# pres_df=pres_df[pres_df['NDC']!='0']

## get list of patients and episodes who were administrated the drug
pres_df=pres_df.dropna(subset=['RxNorm'])
# patients_admin = pres_df[pres_df['RxNorm']==drug_rxnorm][['SUBJECT_ID','HADM_ID']].drop_duplicates()
# write2file(patients_admin,join(drug_single_dir, "patient_episode_list"))
patients_admin = read_data(join(drug_single_dir, "patient_episode_list"),dtype=str)
# patients_admin.head()

In [46]:
print("# of recodes: %d" % len(pres_df))
print("%% of NULL value of NDC: %d%%" % (pres_df['NDC'].isnull().sum() * 100 / len(pres_df)))
print("%% of NULL value of RXNORM: %d%%" % (pres_df['RxNorm'].isnull().sum() * 100 / len(pres_df)))

# of recodes: 3565401
% of NULL value of NDC: 0.000000
% of NULL value of RXNORM: 0.440035


In [7]:
print(len(patients_admin))
print(len(patients_admin.drop_duplicates()))

1852
1852


In [5]:
patients_drugs = left_join(patients_admin,pres_df,['SUBJECT_ID','HADM_ID'])
patients_drugs.head()

,SUBJECT_ID,HADM_ID,NDC
0,12,112213,10019001303
1,12,112213,00045006801
2,12,112213,00074781124
3,12,112213,00338004938
4,12,112213,00002821501


In [6]:
mimic_drug = patients_drugs[['HADM_ID','NDC']]

all_drug['HADM_ID']='remove'
all_drug.head()

mimic_drug = mimic_drug.append(all_drug, sort=True)
mimic_drug['value']=1

mimic_drug_matrix = mimic_drug.pivot_table(index='HADM_ID',columns='NDC',values='value',fill_value=0)
mimic_drug_matrix.head()

mimic_drug_matrix = mimic_drug_matrix.reset_index()
mimic_drug_matrix= mimic_drug_matrix[mimic_drug_matrix['HADM_ID']!='remove']

write2file(mimic_drug_matrix.iloc[:,1:],join(drug_single_dir ,"drugs_presdf"))

In [28]:
def save_items_byfolder(df,file="drugs_presdf",prefix=drug_ade_dir,newfolder=True):
    for subject_id, group in df.groupby(['SUBJECT_ID']):
        if(newfolder):
            create_folder(join(prefix,str(subject_id)))
        
        for hadm_id, subgroup in group.groupby(['HADM_ID']):
            write_dir = join(prefix,str(subject_id),str(hadm_id))
            if(newfolder):
                create_folder(write_dir)
#             [create_folder(join(dis_sum_prefix,str(subject_id),hadm_id)) for hadm_id in hadm_ids]
#             dir = join(dis_sum_prefix,str(subject_id))    
            write2file(subgroup,join(write_dir,file))

In [22]:
save_items_byfolder(patients_drugs)

In [74]:
## all diseases for patients who are administrated the drug
diaglog_df = read_data(join(read_prefix,'DIAGNOSES_ICD'),dtype={'SUBJECT_ID':str, "HADM_ID":str,'ICD9_CODE':str},\
                       usecols=['SUBJECT_ID','HADM_ID','ICD9_CODE']).dropna(subset=['ICD9_CODE']).drop_duplicates()
diaglog_df.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,109,172335,40301
1,109,172335,486
2,109,172335,58281
3,109,172335,5855
4,109,172335,4254


In [33]:
patients_diseases = left_join(patients_admin,diaglog_df,['SUBJECT_ID','HADM_ID'])
patients_diseases.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,12,112213,1570
1,12,112213,57410
2,12,112213,9971
3,12,112213,4275
4,12,112213,99811


In [36]:
mimic_disease = patients_diseases[['HADM_ID','ICD9_CODE']]

all_disease['HADM_ID']='remove'

mimic_disease = mimic_disease.append(all_disease, sort=True)
mimic_disease['value']=1

mimic_disease_matrix = mimic_disease.pivot_table(index='HADM_ID',columns='ICD9_CODE',values='value',fill_value=0)

mimic_disease_matrix = mimic_disease_matrix.reset_index()
mimic_disease_matrix= mimic_disease_matrix[mimic_disease_matrix['HADM_ID']!='remove']

write2file(mimic_disease_matrix.iloc[:,1:],join(drug_single_dir ,"diseases_diagdf"))

In [45]:
def filter_df(df, match_pattern):    
    return df[df['ICD9_CODE'].str.contains(match_pattern)]
E93_df = filter_df(patients_diseases,r'^E93\w+')
save_items_byfolder(patients_diseases,file="diseases_E93",newfolder=False)

In [25]:
# def save_diseases_byfolder(patients_diseases,prefix=drug_ade_dir):
#     for subject_id, group in patients_diseases.groupby(['SUBJECT_ID']):
        
#         for hadm_id, subgroup in group.groupby(['HADM_ID']):
#             write_dir = join(prefix,str(subject_id),str(hadm_id))
# #             [create_folder(join(dis_sum_prefix,str(subject_id),hadm_id)) for hadm_id in hadm_ids]
# #             dir = join(dis_sum_prefix,str(subject_id))    
#             write2file(subgroup,join(write_dir,"diseases_diagdf"))

In [26]:
save_items_byfolder(patients_diseases,file="diseases_diagdf",newfolder=False)

### All new diseases  and prescriptions in that particular episode (using EPISODE_DIAGNOSIS and EPISODE_PRESCRIPTIONS files created)

In [3]:
pres_df = read_data(join(ade_related_prefix,"pres_df"),dtype=str)
pres_df=pres_df.dropna(subset=['RxNorm'])

In [4]:
pres_df.head()

,SUBJECT_ID,HADM_ID,NDC,RxNorm
0,6,107064,00469061711,108513
1,6,107064,00056017275,855334
2,6,107064,00338055002,1658717
3,6,107064,00054829725,310429
4,6,107064,00056016975,855290


In [16]:
# all_drug = list(set(all_drug))
# all_drug = pd.DataFrame({'RxNorm':all_drug},dtype=str)
# write2file(all_drug,join(drug_single_dir,"all_drug"))

# 1. for single drug
# all_drug = read_data(join(drug_single_dir,"all_drug"),dtype=str)
# 2. for all drug
all_drug = read_data(join(ade_related_prefix,"51079075920/single_file","all_drug"),dtype=str)


In [17]:
len(all_drug)

982

In [6]:
def get_binarymatrix(raw_df,extract_fields, all_item):
    df = raw_df[extract_fields]
    all_item_df=all_item
    all_item_df[extract_fields[0]]='remove'
    df = df.append(all_item_df, sort=True)
    df['value']=1

    matrix = df.pivot_table(index='HADM_ID',columns=extract_fields[1],values='value',fill_value=0)

    matrix = matrix.reset_index()
    matrix= matrix[matrix['HADM_ID']!='remove']
    return matrix



In [7]:
## get all diseases
# all_diseases = []
# for folder in ["hospital_course","medical_history","medications_on_admission"]:
#         new_diseases = read_data(join(save_csv_prefix,folder,"ALL/ALL_Disease_icd9"),
#                                  dtype={'SUBJECT_ID':str,'HADM_ID':str}).dropna(subset=['ICD9_CODE'])["ICD9_CODE"].unique().tolist()
#         print(len(new_diseases))
#         all_diseases=all_diseases+new_diseases

# all_diseases=list(set(all_diseases))
# len(all_diseases)
# all_diseases = pd.DataFrame({"ICD9_CODE":all_diseases})
# write2file(all_diseases,join(drug_single_dir,"all_diseases"))


# 1. for single drug
# all_diseases = read_data(join(drug_single_dir,"all_diseases"),dtype=str)
# 2. for all drug
all_diseases = read_data(join(ade_related_prefix,"51079075920/single_file","all_diseases"),dtype=str)

In [16]:
## diseases: check overlapped episodes
epi_dict = {}
for folder in ["hospital_course","medical_history","medications_on_admission"]:
#     concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
#     drug_df = getdrug(concatcsv)
    
# ["hospital_course","medications_on_admission","medical_history"]:
    cols=["SUBJECT_ID","HADM_ID","START INDEX","END INDEX","ASSERTION","SNOMEDCT_US","ICD9_CODE"]
    new_disease_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Disease_icd9"),
                               dtype={'SUBJECT_ID':str,'HADM_ID':str}).dropna(subset=['ICD9_CODE'])[["HADM_ID","SNOMEDCT_US","ICD9_CODE"]]

    epi_dict[folder]=set(new_disease_df.HADM_ID.unique())
# print("{:}: {:,d} episodes, {:,d} SNOMEDCT_US, {:,d} ICD9_CODE, {:,d} Rows".format(folder,
#     len(new_disease_df.HADM_ID.unique()), len(new_disease_df.SNOMEDCT_US.unique()),len(new_disease_df.ICD9_CODE.unique()), len(new_disease_df)))




import itertools
folders = ["hospital_course","medical_history","medications_on_admission"]
# from itertools import combinations
for comb in set(itertools.combinations(folders, 2)):
    print("{:} ∩ {:}: {:,d} episodes".format(comb[0],comb[1],len(epi_dict[comb[0]]&epi_dict[comb[1]])))
# epi_dict['hospital_course']
# print("{:}: {:,d} episodes".format
print("{:} ∩ {:} ∩ {:}: {:,d} episodes".format(
    folders[0],folders[1],folders[2],len(epi_dict[folders[0]] & epi_dict[folders[1]] & epi_dict[folders[2]])))


medical_history ∩ medications_on_admission: 7,328 episodes
hospital_course ∩ medical_history: 3,760 episodes
hospital_course ∩ medications_on_admission: 3,342 episodes
hospital_course ∩ medical_history ∩ medications_on_admission: 2,817 episodes


In [6]:
## diseases
for folder in ["hospital_course","medical_history","medications_on_admission"]:
#     concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
#     drug_df = getdrug(concatcsv)
    
# ["hospital_course","medications_on_admission","medical_history"]:
    cols=["SUBJECT_ID","HADM_ID","START INDEX","END INDEX","ASSERTION","SNOMEDCT_US","ICD9_CODE"]
    new_disease_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Disease_icd9"),
                               dtype={'SUBJECT_ID':str,'HADM_ID':str}).dropna(subset=['ICD9_CODE'])[["HADM_ID","SNOMEDCT_US","ICD9_CODE"]]

    print("{:}: {:,d} episodes, {:,d} SNOMEDCT_US, {:,d} ICD9_CODE, {:,d} Rows".format(folder,
        len(new_disease_df.HADM_ID.unique()), len(new_disease_df.SNOMEDCT_US.unique()),len(new_disease_df.ICD9_CODE.unique()), len(new_disease_df)))


    
#     ##1. inner join if single drug 2. no inner join for all records
# #     patients_diseases = inner_join(patients_admin,new_disease_df,["SUBJECT_ID","HADM_ID"])
# #     patients_diseases = patients_diseases[['HADM_ID','ICD9_CODE']]
#     patients_diseases = new_disease_df[['HADM_ID','ICD9_CODE']]
        
#     matrix = get_binarymatrix(patients_diseases,extract_fields =['HADM_ID','ICD9_CODE'], all_item=all_diseases)   
#     print(matrix.shape)
# #     write2file(matrix,join(drug_single_dir ,"%s_diseaseMatrix"%folder))
# #     save_items_byfolder(patients_diseases,file="diseases_%s"%folder,newfolder=False)

hospital_course: 4,772 episodes, 496 SNOMEDCT_US, 495 ICD9_CODE, 9,472 Rows
medical_history: 10,974 episodes, 784 SNOMEDCT_US, 783 ICD9_CODE, 41,341 Rows
medications_on_admission: 9,121 episodes, 985 SNOMEDCT_US, 984 ICD9_CODE, 40,406 Rows


In [24]:
all_drug = []
for folder in ["hospital_course","medical_history","medications_on_admission"]:
#     concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
#     drug_df = getdrug(concatcsv)
    
# ["hospital_course","medications_on_admission","medical_history"]:
    cols=["SUBJECT_ID","HADM_ID","RxNorm"]
    new_drug_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Drug_NDCs"),
                            dtype={'SUBJECT_ID':str,'HADM_ID':str})[cols]

    print(len(new_drug_df['RxNorm'].unique()))
    all_drug = all_drug+list(new_drug_df['RxNorm'].unique())

884
2252
3211


In [27]:
# all_drug = list(set(all_drug))
all_drug_df = pd.DataFrame({"RxNorm":all_drug})
write2file(all_drug_df,join(ade_related_prefix,"all_drug"))

In [23]:
for folder in ["hospital_course","medical_history","medications_on_admission"]:
#     concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
#     drug_df = getdrug(concatcsv)
    
# ["hospital_course","medications_on_admission","medical_history"]:
    cols=["SUBJECT_ID","HADM_ID","RxNorm"]
    new_drug_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Drug_NDCs"),
                            dtype={'SUBJECT_ID':str,'HADM_ID':str})[cols]

    print("{:}: {:,d} episodes, {:,d} RxNorm codes, {:,d} Rows".format(folder,
        len(new_drug_df.HADM_ID.unique()), len(new_drug_df.RxNorm.unique()), len(new_drug_df)))
    
    
#     ## use for one drug; comment if for all episodes
# #     patients_drugs = inner_join(patients_admin,new_drug_df,["SUBJECT_ID","HADM_ID"])[['HADM_ID','RxNorm']]
#     patients_drugs = new_drug_df[['HADM_ID','RxNorm']]
    
#     matrix = get_binarymatrix(patients_drugs,extract_fields =['HADM_ID','RxNorm'],all_item=all_drug_df)    
#     print(matrix.shape)
#     write2file(matrix,join(drug_single_dir ,"%s_drugMatrix"%folder))
##     save_items_byfolder(patients_drugs,file="drugs_%s"%folder,newfolder=False)

hospital_course: 10,106 episodes, 884 RxNorm codes, 55,755 Rows
medical_history: 8,722 episodes, 2,252 RxNorm codes, 38,572 Rows
medications_on_admission: 12,174 episodes, 3,211 RxNorm codes, 114,049 Rows


In [18]:
## diseases: check overlapped episodes
epi_dict = {}
for folder in ["hospital_course","medical_history","medications_on_admission"]:

    cols=["SUBJECT_ID","HADM_ID","RxNorm"]
    new_drug_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Drug_NDCs"),
                            dtype={'SUBJECT_ID':str,'HADM_ID':str})[cols]


    epi_dict[folder]=set(new_drug_df.HADM_ID.unique())
# print("{:}: {:,d} episodes, {:,d} SNOMEDCT_US, {:,d} ICD9_CODE, {:,d} Rows".format(folder,
#     len(new_disease_df.HADM_ID.unique()), len(new_disease_df.SNOMEDCT_US.unique()),len(new_disease_df.ICD9_CODE.unique()), len(new_disease_df)))




import itertools
folders = ["hospital_course","medical_history","medications_on_admission"]
# from itertools import combinations
for comb in set(itertools.combinations(folders, 2)):
    print("{:} ∩ {:}: {:,d} episodes".format(comb[0],comb[1],len(epi_dict[comb[0]]&epi_dict[comb[1]])))
# epi_dict['hospital_course']
# print("{:}: {:,d} episodes".format
print("{:} ∩ {:} ∩ {:}: {:,d} episodes".format(
    folders[0],folders[1],folders[2],len(epi_dict[folders[0]] & epi_dict[folders[1]] & epi_dict[folders[2]])))


medical_history ∩ medications_on_admission: 7,378 episodes
hospital_course ∩ medical_history: 5,947 episodes
hospital_course ∩ medications_on_admission: 8,622 episodes
hospital_course ∩ medical_history ∩ medications_on_admission: 5,245 episodes


## What is the total number of episodes from which you extracted all the information? 

Can we see the following statistics from the data you have extracted:
1. Number of episodes where there are new diseases. New diseases = diseases that are not in “past medical history” but present in “hospital course” .
> Are there episodes with diseases in the (full) diagnosis table that are not there in both “past medical history” and “hospital course”? We should check these cases – they may be due to loss of information or errors in any step - MIMIC or CLAMP or in our data processing. Please list these episodes and for each episode, the list of diseases not found in both “past medical history” and “hospital course” (in a file).
2. What is the distribution of new diseases – list the names of the new diseases, ICD codes, and number of episodes in a file.
3. Number of episodes where there are new drugs. New drugs =  drugs that are not in “medications on admission” but present in “hospital course”.
> Are there episodes with drugs  in the (full) prescriptions table that are not there in both “medications on admission” and “hospital course”? Please list these episodes and for each episode, the list of drugs not found in both “medications on admission” and “hospital course” (in a file).
4. What is the distribution of new drugs – list the names of the new drugs, RxNorm codes, and number of episodes in a file.

In [9]:
def mat_noEmpty(mat,axis_num=0):
    if(axis_num):
        ## drop row
        mat = mat[(mat!=0).any(axis=1)]
    else:
        ## drop column
        mat = mat.loc[:,(mat!=0).any(axis=0)]
    return mat

def mat_df(mat,item):
    df1 = pd.DataFrame(mat.set_index('HADM_ID').stack()).reset_index()
    df2 = df1[df1[0]!=0].drop(0,1)
    df2.columns=["HADM_ID",item]
    return df2

In [20]:
## Revised version: New items per episode
def get_new_item(mat1, mat2,map_dict,full_df, epi_field="HADM_ID",item="ICD9_CODE", name="NAME"):
#     both_episodes = set(mat1[epi_field].unique()).union(set(mat2[epi_field].unique()))
    common_episodes = set(mat1[epi_field].unique()).intersection(set(mat2[epi_field].unique()))
    
    df3 = mat_df(mat1,item).append(mat_df(mat2,item)).drop_duplicates()
    df4 = full_df.merge(df3, on=["HADM_ID",item], how='left', indicator=True)
    df_missEPi = df4[df4['_merge']=='left_only'].drop("_merge",1)
#     write2file(df_missEPi,join(drug_single_dir,"missing_episode%s"%item))

    print("Matrix including HADM_ID as the first column")
    print((mat_noEmpty(mat2).shape))
    print((mat_noEmpty(mat1).shape))

    common_episodes_df = pd.DataFrame({epi_field:list(common_episodes)})
    mat1_comepi = mat_noEmpty(left_join(common_episodes_df,mat1,epi_field))
    mat2_comepi = mat_noEmpty(left_join(common_episodes_df,mat2,epi_field))
    print((mat2_comepi.shape))
    print((mat1_comepi.shape))

    ## mat1: hospital course
    ## List of new codes are based on collection of codes in hospital course
    mat2_comepi_standard=mat2_comepi.reindex(list(mat1_comepi.columns),axis=1).fillna(0)
    
    # find new item
    mat1_newitems = (mat1_comepi.set_index("HADM_ID")>mat2_comepi_standard.set_index("HADM_ID"))*1
    
    ## remove empty rows (hospital stays) and columns (diseases)
    mat1_newitems = mat_noEmpty(mat1_newitems,axis_num=1)
    mat1_newitems = mat_noEmpty(mat1_newitems,axis_num=0)
    print("Matrix without HADM_ID as the first column")
    print(mat1_newitems.shape)
    write2file(mat1_newitems.reset_index(),join(drug_single_dir,"allepis_new%s"%item))
    
    ## reset index (hadm_id) as the first column
    mat1_newitems_distri = pd.DataFrame(mat1_newitems.sum()).reset_index()
    mat1_newitems_distri.columns=[item, "NUM_OF_EPISODES"]
    mat1_newitems_distri[name]=mat1_newitems_distri[item].apply(lambda x: map_dict.get(x,None))
    
    print("Number of espisodes containing new items: %d" % len(mat1_newitems))
    write2file(mat1_newitems_distri,join(drug_single_dir,"distribution_of_new%s"%item))
    


In [3]:
## Old version: new items: collections of items across all episodes
def get_new_item(mat1, mat2,map_dict,full_df, epi_field="HADM_ID",item="ICD9_CODE", name="NAME"):
#     both_episodes = set(mat1[epi_field].unique()).union(set(mat2[epi_field].unique()))
    common_episodes = set(mat1[epi_field].unique()).intersection(set(mat2[epi_field].unique()))
    
    df3 = mat_df(mat1,item).append(mat_df(mat2,item)).drop_duplicates()
    df4 = full_df.merge(df3, on=["HADM_ID",item], how='left', indicator=True)
    df_missEPi = df4[df4['_merge']=='left_only'].drop("_merge",1)
    write2file(df_missEPi,join(drug_single_dir,"missing_episode%s"%item))

    print((mat_noEmpty(mat2).shape))
    print((mat_noEmpty(mat1).shape))

    common_episodes_df = pd.DataFrame({epi_field:list(common_episodes)})
    mat1_comepi = mat_noEmpty(left_join(common_episodes_df,mat1,epi_field))
    mat2_comepi = mat_noEmpty(left_join(common_episodes_df,mat2,epi_field))
    print((mat2_comepi.shape))
    print((mat1_comepi.shape))
    
    new_items = set(mat1_comepi.columns)-set(mat2_comepi.columns)
    print("Number of new items: %d"%len(new_items))
    
    mat1_comepi=mat1_comepi.set_index("HADM_ID")
    
    mat1_newitems = mat_noEmpty(mat1_comepi[list(new_items)],axis_num=1)
    mat1_newitems_distri = pd.DataFrame(mat1_newitems.sum()).reset_index()
    mat1_newitems_distri.columns=[item, "NUM_OF_EPISODES"]
    mat1_newitems_distri[name]=mat1_newitems_distri[item].apply(lambda x: map_dict.get(x,None))
    
    print("Number of espisodes containing new items: %d" % 
          len(mat1_newitems))
    write2file(mat1_newitems_distri,join(drug_single_dir,"distribution_of_new%s"%item))
       

In [21]:
## revised: get new diseases
code_name_map = read_data(join(read_prefix,"D_ICD_DIAGNOSES"),dtype=str).set_index("ICD9_CODE")['SHORT_TITLE'].to_dict()
diaglog_df = read_data(join(read_prefix,'DIAGNOSES_ICD'),dtype={'SUBJECT_ID':str, "HADM_ID":str,'ICD9_CODE':str},\
                       usecols=['SUBJECT_ID','HADM_ID','ICD9_CODE']).dropna(subset=['ICD9_CODE']).drop_duplicates()

get_new_item(read_data(join(drug_single_dir,"hospital_course_diseaseMatrix"),dtype={"HADM_ID":str}),
             read_data(join(drug_single_dir,"medical_history_diseaseMatrix"),dtype={"HADM_ID":str}),
             map_dict=code_name_map,full_df=diaglog_df)

Matrix including HADM_ID as the first column
(10974, 784)
(4772, 496)
(3760, 557)
(3760, 441)
Matrix without HADM_ID as the first column
(3626, 432)
Number of espisodes containing new items: 3626


In [32]:
## old version : get new diseases collections 
code_name_map = read_data(join(read_prefix,"D_ICD_DIAGNOSES"),dtype=str).set_index("ICD9_CODE")['SHORT_TITLE'].to_dict()
diaglog_df = read_data(join(read_prefix,'DIAGNOSES_ICD'),dtype={'SUBJECT_ID':str, "HADM_ID":str,'ICD9_CODE':str},\
                       usecols=['SUBJECT_ID','HADM_ID','ICD9_CODE']).dropna(subset=['ICD9_CODE']).drop_duplicates()

get_new_item(read_data(join(drug_single_dir,"hospital_course_diseaseMatrix"),dtype={"HADM_ID":str}),
             read_data(join(drug_single_dir,"medical_history_diseaseMatrix"),dtype={"HADM_ID":str}),
             map_dict=code_name_map,full_df=diaglog_df)

(10974, 784)
(4772, 496)
(3760, 557)
(3760, 441)
Number of new items: 162
Number of espisodes containing new items: 399


In [22]:
## get new drugs
rxnorm_name_map = read_data(join(ade_related_prefix,"RxNorm_DRUGNAME"),dtype=str).set_index("RxNorm")['DRUG'].to_dict()


pres_df = read_data(join(ade_related_prefix,"pres_df"),dtype=str)
pres_df=pres_df.dropna(subset=['RxNorm'])
pres_df.head()

,SUBJECT_ID,HADM_ID,NDC,RxNorm
0,6,107064,00469061711,108513
1,6,107064,00056017275,855334
2,6,107064,00338055002,1658717
3,6,107064,00054829725,310429
4,6,107064,00056016975,855290


In [23]:
get_new_item(read_data(join(drug_single_dir,"hospital_course_drugMatrix"),dtype={"HADM_ID":str}),
             read_data(join(drug_single_dir,"medications_on_admission_drugMatrix"),dtype={"HADM_ID":str}),
             rxnorm_name_map,full_df=pres_df[['HADM_ID','RxNorm']],epi_field="HADM_ID",item="RxNorm", name="NAME")

Matrix including HADM_ID as the first column
(12174, 3212)
(10106, 885)
(8622, 2855)
(8622, 667)
Matrix without HADM_ID as the first column
(8511, 621)
Number of espisodes containing new items: 8511


## Get  SIDER+ EPISODE_DIAGNOSES

In [3]:
sider_fulldf = read_data(join(ade_related_prefix,"sider4_v1"),dtype=str)[["RxNorm","ICD9_CODE"]].drop_duplicates()

In [4]:
sider_fulldf.head()

,RxNorm,ICD9_CODE
0,1085756,78936
1,1085752,78936
2,105398,78936
3,208680,78936
4,208712,78936
